In [3]:
import img2pdf
import glob
from fpdf import FPDF
from PIL import Image
import os
import ntpath
import ocrmypdf 
import numpy as np
import re
import pdfplumber
import pandas as pd
import time
import cv2
from matplotlib import pyplot as plt
from pdf2image import convert_from_path
pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/n01/data/nlp_aeac/mostofa_env/mo_venv/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [57]:

#https://stackoverflow.com/questions/28816046/
#displaying-different-images-with-actual-size-in-matplotlib-subplot
def display(im_path):
    dpi = 80
    im_data = plt.imread(im_path)

    height, width  = im_data.shape[:2]
    
    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(im_data, cmap='gray')

    plt.show()
    

    
################ GRAY SCALE CONVERSION ###########    
def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)    



################ Noise Removal ###########
def noise_removal(image):
    import numpy as np
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.medianBlur(image, 3)
    return (image)

def morph_open(image):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,2))
    morphology_img = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel,iterations=1)
    return (morphology_img)
#     plt.imshow(morphology_img,'Greys_r')

########### Dilation and Erosion ##########
#######
#######
#######
########### Making the front THIN ########
def thin_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2),np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return (image)



########### Making the front THICK ########
def thick_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2),np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return (image)




########### DETECTING SKEW ########
def getSkewAngle(cvImage) -> float:
    # Prep image, copy, convert to gray scale, blur, and threshold
    newImage = cvImage.copy()
    gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Apply dilate to merge text into meaningful lines/paragraphs.
    # Use larger kernel on X axis to merge characters into single line, cancelling out any spaces.
    # But use smaller kernel on Y axis to separate between different blocks of text
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=2)

    # Find all contours
    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    for c in contours:
        rect = cv2.boundingRect(c)
        x,y,w,h = rect
        cv2.rectangle(newImage,(x,y),(x+w,y+h),(0,255,0),2)

    # Find largest contour and surround in min area box
    largestContour = contours[0]
    print (len(contours))
    minAreaRect = cv2.minAreaRect(largestContour)
#     cv2.imwrite("temp/boxes.jpg", newImage)
    # Determine the angle. Convert it to the value that was originally used to obtain skewed image
    angle = minAreaRect[-1]
    if angle < -45:
        angle = 90 + angle
    return -1.0 * angle




############## Rotate the image around its center ###############
def rotateImage(cvImage, angle: float):
    newImage = cvImage.copy()
    (h, w) = newImage.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    newImage = cv2.warpAffine(newImage, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return newImage



############### Deskew image ###############
def deskew(cvImage):
    angle = getSkewAngle(cvImage)
    return rotateImage(cvImage, -1.0 * angle)



############### REMMOVING BORDERS ###############
def remove_borders(image):
    contours, heiarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntsSorted = sorted(contours, key=lambda x:cv2.contourArea(x))
    cnt = cntsSorted[-1]
    x, y, w, h = cv2.boundingRect(cnt)
    crop = image[y:y+h, x:x+w]
    return (crop)

################## CONVERT PDF TO LINES ##################

def Pdf_to_lines(pdf_file_path):
    raw_lines= {}
    total_line_cnt=0
    with pdfplumber.open(pdf_file_path) as pdf:
        page_count=pdf.pages[:]
#         print(len(page_count))
        for i in range (0,len(page_count)):
            page =pdf.pages[i]
            page_nm=i+1
    #         print(page_nm)
    #         print(page.page_number)
            text = page.extract_text()
            raw = text.split('\n')
            for i in range(0, len(raw)):
                line_nm=i+1
                total_line_cnt=total_line_cnt+1
                k= "Page-"+str(page_nm)+"_Line-"+str(line_nm)
#                 k= "Page-"+str(page_nm)+"_Line-"+str(line_nm)+"_TL-"+str(total_line_cnt)
                raw_lines[k]= raw[i]
    return raw_lines



############# Make PDF Searchable Lines #############


timestr = time.strftime("%m%d%Y-%H%M%S")

def PDF_searchable_lines(input_file_dir, file_name):
#     PDF_filename = str(input_file)+str('.pdf')
#     input_dir = "/n01/data/nlp_aeac/mostofa_env/INPUT/CLEAN_PDF/"
#     input_file_dir= str(input_dir)+ str(PDF_filename)
    output_loc = "/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/searchable_pdf/"
#     timestr = time.strftime("%m%d%Y-%H%M%S")
    # print(PDF_filename)
#     filename=PDF_filename.rsplit(".",1)[:-1]
    # print(str(filename[0]))
#     csv_path='/n01/data/nlp_aeac/mostofa_env/INPUT/CONVERTED_CSV_OUTPUT/'
    output_name= output_loc+file_name[:-4]+'_'+str(timestr)+'.pdf'
    ocrmypdf.ocr(input_file_dir, output_name,rotate_pages=True, deskew=True, force_ocr=True)
    output_lines= Pdf_to_lines(output_name)
    return output_lines

########################## DATE EXTRACTION ################################


def date_detection(input_line_string):
    pref_num = re.compile(r'(\d{1,2}\/\d{1,2}\/\d{4}|\d{1,2}\/\d{1,2}\/\d{2}|\d{1,2}\-\d{1,2}\-\d{4}|\d{1,2}\-\d{1,2}\-\d{2}|\d{1,2}\d{1,2}\d{4}|\d{1,2}\d{1,2}\d{1,4}_(\d+)_)')
    m = pref_num.search(input_line_string)
    
    if m != None:
#         print(m.group(1))
#         print(m.string)
        return m.group(1)
    else:
        return "N/A"
        
# date_detection(l_s)




######################## DESCRIPTION EXTRACTION ##############################


# l_s='0250  |022621|  7955  |250  DEXTROSE  5  %  AND  0.9  %  SODIUM  1  15.00 '

def description_detection(input_line_string):
    loc=[]
    terms=[]
    pattern = r'([A-Za-z]+[\w@]*)'
    for match in re.finditer(pattern, input_line_string):
        s = match.start()
        e = match.end()
#         print('String match : ',(input_line_string[s:e], s, e))
        terms.append(input_line_string[s:e])
        loc.append(s)
        loc.append(e)
    split_loc=[loc[0], loc[-1]]
    truncated_text = input_line_string[loc[0]:loc[-1]]
#     print(truncated_text)
    return truncated_text

        





############################# DOLLAR EXTRACTION #####################


# description_detection(l_s)
# dollar_type_1 = '\d{1,2}[,\.]\d{1,3}[,\.]\d{2}'
# dollar_type_2 = '\d{1,1}[,\.]\d{1,3}[,\.]\d{2}'
# dollar_type_3 = '\d{1,3}[,\.]\d{1,3}[,\.]\d{2}'
# dollar_type_4 = '\d{1,2}[,\.]\d{1,3}[,\.]\d{1}'
# dollar_type_5 = '\d{1,1}[,\.]\d{1,3}[,\.]\d{1}'
# dollar_type_6 = '\d{1,3}[,\.]\d{1,3}[,\.]\d{1}'
# dollar_type_3 = '\d+[.]\d+'

# l_s='Page-1_Line-34 2500000  HEMOSTATIC  MATRIX $.30'

def dollar_detection(input_line_string):
    pref_num = re.findall('(\d{1,2}[,\.]\d{1,3}[,\.]\d{2}|d{1,1}[,\.]\d{1,3}[,\.]\d{2}|\d{1,3}[,\.]\d{1,3}[,\.]\d{2}|\d{1,2}[,\.]\d{1,3}[,\.]\d{1}|\d{1,1}[,\.]\d{1,3}[,\.]\d{2}|\d{1,3}[,\.]\d{1,3}[,\.]\d{1}|\d+[,\.]\d+|_(\d+)_)', input_line_string, re.DOTALL)
#     m = pref_num.search(input_line_string)
    match_length=len(pref_num)
    
   
    if match_length > 0:
        m=pref_num[-1]
#         print(m.group(1))
        s =m[0]
        if s[-2]==',':
            s = s.replace(',', '.')
        if s[-3]==',':
            s = s.replace(',', '.')
        s1 = s.replace(',', '')
#         print(s1)
        n=float(s1)
        return n
    else:
#         print('NA')
        return 0
        
# quant=dollar_detection(l_s)
# print(quant)

########################## DESCRIPTION EXTRACTION NEW_TEST ############################


# l_s='0250  |022621|  7955  |250  DEXTROSE  5  %  AND  0.9  %  SODIUM  1  15.00 '

def description_detection_1(input_line_string):
    loc=[]
    terms=[]
    pattern = r'([A-Za-z]+[\w@]*)'
    pref_num = re.compile(r'([A-Za-z]+[\w@]*)')
    m = pref_num.search(input_line_string)
    if m != None:
        for match in re.finditer(pattern, input_line_string):
            s = match.start()
            e = match.end()
    #         print('String match : ',(input_line_string[s:e], s, e))
            terms.append(input_line_string[s:e])
            loc.append(s)
            loc.append(e)
        split_loc=[loc[0], loc[-1]]
        truncated_text = input_line_string[loc[0]:loc[-1]]
    #     print(truncated_text)
        return truncated_text
    else:
        return 'No Description'

        
# description_detection(l_s)




################# DOLLAR EXTRACTION NEW_TEST ##########################




# dollar_type_1 = '\d{1,2}[,\.]\d{1,3}[,\.]\d{2}'
# dollar_type_2 = '\d{1,1}[,\.]\d{1,3}[,\.]\d{2}'
# dollar_type_3 = '\d{1,3}[,\.]\d{1,3}[,\.]\d{2}'
# dollar_type_4 = '\d{1,2}[,\.]\d{1,3}[,\.]\d{1}'
# dollar_type_5 = '\d{1,1}[,\.]\d{1,3}[,\.]\d{1}'
# dollar_type_6 = '\d{1,3}[,\.]\d{1,3}[,\.]\d{1}'
# dollar_type_3 = '\d+[.]\d+'

# l_s='Page-1_Line-34 2500000  HEMOSTATIC  MATRIX $.30'

def dollar_detection_1(input_line_string):
    pref_num = re.compile(r'(\d{1,2}[,\.]\d{1,3}[,\.]\d{2}|d{1,1}[,\.]\d{1,3}[,\.]\d{2}|\d{1,3}[,\.]\d{1,3}[,\.]\d{2}|\d{1,2}[,\.]\d{1,3}[,\.]\d{1}|\d{1,1}[,\.]\d{1,3}[,\.]\d{2}|\d{1,3}[,\.]\d{1,3}[,\.]\d{1}|\d+[.]\d+|_(\d+)_)')
    m = pref_num.search(input_line_string)
    
    if m != None:
#         print(m.group(1))
        s =m.group(1)
        if s[-2]==',':
            s = s.replace(',', '.')
        if s[-3]==',':
            s = s.replace(',', '.')
        s1 = s.replace(',', '')
#         print(s1)
        n=float(s1)
        return n
    else:
#         print('NA')
        return 'NaN'
        
# quant=dollar_detection(l_s)
# print(quant)



############ Search Term to DataFrame and Total Amount #########




def Search_in_PDF(term):
    search_term = str(term)
    t = []
    sl =[]
    date_l =[]
    des_l=[]
    search_l=[]
    position_l =[]
    flag_l=[]
    line_l=[]
    for key, val in clean_searchable_pdf_lines.items():
        if search_term in val:
            sd = key + " " + " : " + val
            line_l.append(sd)
            sl.append(re.split(r'\s+', sd))

            date_strip =date_detection(val)
            description = description_detection_1(val)
            dollar=dollar_detection(val)

            if dollar!=0:
                t.append(dollar)
                flag_l.append("")
            else:
                t.append(0)
                flag_l.append("Check the Amount")
            date_l.append(date_strip)
            des_l.append(description)
            search_l.append(search_term)
            position_l.append(key)

    print(t)
    total_amount =np.sum(t)
    print("Total Amount : ",total_amount) 
    # print(sl)
    clean_data= pd.DataFrame(
        {'Position': position_l,
        'Search_Term': search_l,
         'Date': date_l,
         'Description': des_l,
         'Amount': t,
         'FLAG' :flag_l,
         'Line_info' : line_l
        })
    return t, total_amount, clean_data




############## Save the Dataframe ###############




def save_SearchTerm_DF_to_CSV(filename, search_key, df, timestr):
    csv_path='/n01/data/nlp_aeac/mostofa_env/INPUT/OUTPUT_CSV_TEMP/'
    raw_output_name= csv_path+"RAW_Search_"+str(search_key)+"_"+filename+'_'+str(timestr)+'.csv'
    print('Saved as: --',raw_output_name)
    df.to_csv(raw_output_name, sep=',', index=False)

    



############## Search Term to DataFrame and Total Amount ###############

def PDF_to_DF(start_df,end_df):

    t = []
    sl =[]
    date_l =[]
    des_l=[]
    line_text=[]
#     search_l=[]
    position_l =[]
    flag_l=[]
    
            
        
    for key, val in clean_searchable_pdf_lines.items():
        start_TL = key.split('_')[2]
        start_TL=start_TL.split('-')[1]
        start_TL=int(start_TL)
        if start_TL in range(start_df,end_df):
            print(start_TL)
            print(val)
            line_text.append(val)
            sd = key + " " + val
            sl.append(re.split(r'\s+', sd))

            date_strip =date_detection(val)
            description = description_detection_1(val)
            dollar=dollar_detection_1(val)

            if dollar!=0:
                t.append(dollar)
                flag_l.append("")
            else:
                t.append(0)
                flag_l.append("Check the Amount")
            date_l.append(date_strip)
            des_l.append(description)
            position_l.append(key)

#     print(t)
#     total_amount =np.sum(t)
#     print("Total Amount : ",total_amount) 
    # print(sl)
    clean_data= pd.DataFrame(
        {'Position': position_l,
         'Date': date_l,
         'Text': line_text,
         'Description': des_l,
         'Amount': t,
         'FLAG' :flag_l
        })
    return t, clean_data





def trigger_p(image):
    hist = cv2.calcHist([image],[0],None,[255],[0,255])
    within = []
    for i in range(len(hist)):
        x,y = np.split(hist,[i])
        x1 = np.sum(x)/(image.shape[0]*image.shape[1])
        y1 = np.sum(y)/(image.shape[0]*image.shape[1])
        x2 = np.sum([ j*t for j,t in enumerate(x)])/np.sum(x)
        y2 = np.sum([ j*t for j,t in enumerate(y)])/np.sum(y)
        x3 = np.sum([ (j-x2)**2*t for j,t in enumerate(x)])/np.sum(x)
        x3 = np.nan_to_num(x3)
        y3 = np.sum([ (j-y2)**2*t for j,t in enumerate(y)])/np.sum(y)
        y3 = np.nan_to_num(y3)
        within.append(x1*x3 + y1*y3)
    m = np.argmin(within)
    print(m)
    return m

#############################################################
##############################################################
########### PDF TO IMAGE PROCESSING #######################
#####
####

def pdf_to_image(s):
    upload_loc='/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_uploads/'
    outputLoc = '/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_processed/'
    outputLoc_1 = '/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/'
    filename = s.replace(upload_loc,"")
    print(filename)
    img_name=filename[:-4]
    print(img_name)
    inputLoc = s
    ocrLoc = outputLoc+filename
    ocrLoc_1 = outputLoc_1+filename
    if not os.path.exists(ocrLoc_1):
        os.makedirs(ocrLoc_1)
        os.makedirs(ocrLoc)
    dpi=200
    pages = convert_from_path(inputLoc ,dpi )
    print("New File Transfer........")
    for i in range(len(pages)):
        page = pages[i]
        page.save('{}/{}_{}.jpg'.format(ocrLoc_1,img_name,i), 'JPEG')
        new_im=ocrLoc_1+"/"+img_name+"_"+str(i)+".jpg"
        print(new_im)
        img = cv2.imread(new_im)
#         img=deskew(img)
        gray_image = grayscale(img)
        t_p = trigger_p(gray_image)
        if (t_p < 128):
            gray_image = noise_removal(gray_image)
            print('NOISE REMOVAL DONE')
            
#         morph=morph_open(gray_image)
        kernel_dilation = np.ones((1,1), np.uint8)
#         gray_image=deskew(gray_image)
        ret, cv_otsu_binary = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        dilation = cv2.dilate(cv_otsu_binary, kernel_dilation, iterations=1)
        cv2.imwrite('{}/{}_{}.jpg'.format(ocrLoc,img_name,i), dilation)
    return ocrLoc, filename


#############################################################
##############################################################
########### IMAGE TO PDF PROCESSING #######################
#####
####


def image_to_pdf(input_folder_loc, file_name):
    OutPutLoc= '/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/image_processed_pdf/'
    img_folder_list= input_folder_loc+"/*.jpg"
    imagelist = sorted(glob.glob(img_folder_list), key=os.path.getmtime)
    pdf= FPDF()
    out_pdf_name= OutPutLoc+file_name
    with open(out_pdf_name,"wb") as f:
        f.write(img2pdf.convert(imagelist))
    return out_pdf_name

In [56]:
########### PDF UPLOAD AS INPUT #######################   

s= '/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_uploads/CAMC_IB_Dana Keen_Claim # 21755930880.pdf'
# s= '/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_uploads/Betsy Stewart_Claim # 21740462937.pdf'

########### PDF TO IMAGE PROCESSING #######################    
processed_image_loc, pdf_name = pdf_to_image(s)
print(processed_image_loc)
print(pdf_name)
########### IMAGE TO PDF PROCESSING #######################
processed_pdf = image_to_pdf(processed_image_loc, pdf_name)
print(processed_pdf)
########### PROCESSED PDF TO SEARCHABLE PDF #######################
clean_searchable_pdf_lines= PDF_searchable_lines(processed_pdf, pdf_name )


########### INPUT SEARCH TERM INTO SEARCHABLE PDF #######################
List_check=["0272", "250"]
search_word = "0278"

cost_list_1, total_cost_1, search_df_1 = Search_in_PDF(search_word)
search_df_1

CAMC_IB_Dana Keen_Claim # 21755930880.pdf
CAMC_IB_Dana Keen_Claim # 21755930880
New File Transfer........
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/CAMC_IB_Dana Keen_Claim # 21755930880.pdf/CAMC_IB_Dana Keen_Claim # 21755930880_0.jpg


/n01/data/nlp_aeac/mostofa_env/mo_venv/lib/python3.7/site-packages/ipykernel_launcher.py:474: RuntimeWarning: invalid value encountered in double_scalars
/n01/data/nlp_aeac/mostofa_env/mo_venv/lib/python3.7/site-packages/ipykernel_launcher.py:476: RuntimeWarning: invalid value encountered in double_scalars


126
NOISE REMOVAL DONE
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/CAMC_IB_Dana Keen_Claim # 21755930880.pdf/CAMC_IB_Dana Keen_Claim # 21755930880_1.jpg
126
NOISE REMOVAL DONE
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/CAMC_IB_Dana Keen_Claim # 21755930880.pdf/CAMC_IB_Dana Keen_Claim # 21755930880_2.jpg
126
NOISE REMOVAL DONE
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/CAMC_IB_Dana Keen_Claim # 21755930880.pdf/CAMC_IB_Dana Keen_Claim # 21755930880_3.jpg
126
NOISE REMOVAL DONE
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/CAMC_IB_Dana Keen_Claim # 21755930880.pdf/CAMC_IB_Dana Keen_Claim # 21755930880_4.jpg
126
NOISE REMOVAL DONE
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/CAMC_IB_Dana Keen_Claim # 21755930880.pdf/CAMC_IB_Dana Keen_Claim # 21755930880_5.jpg
126
NOISE REMOVAL DONE
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/CAMC_IB_Dana Keen_Claim # 21755930880.pdf/CAMC_IB_Dana Keen_Claim # 2175

Scanning contents:   0%|                                | 0/9 [00:00<?, ?page/s]

/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/image_processed_pdf/CAMC_IB_Dana Keen_Claim # 21755930880.pdf


Scanning contents: 100%|███████████████████████| 9/9 [00:00<00:00, 133.37page/s]
OCR:   6%|█▉                                | 0.5/9.0 [00:06<01:44, 12.30s/page][tesseract] lots of diacritics - possibly poor OCR
OCR:  39%|█████████████▏                    | 3.5/9.0 [00:09<00:09,  1.70s/page][tesseract] lots of diacritics - possibly poor OCR
OCR:  94%|████████████████████████████████  | 8.5/9.0 [00:11<00:00,  1.79page/s][tesseract] lots of diacritics - possibly poor OCR
OCR: 100%|██████████████████████████████████| 9.0/9.0 [00:11<00:00,  1.27s/page]
PDF/A conversion: 100%|█████████████████████████| 9/9 [00:02<00:00,  3.36page/s]
JPEGs: 0image [00:00, ?image/s]
JBIG2: 0item [00:00, ?item/s]


[252.0, 536.0, 103.0, 282.0, 207.0]
Total Amount :  1380.0


,Position,Search_Term,Date,Description,Amount,FLAG,Line_info
0,Page-6_Line-21,0278,021221,WIRE FLEXIBLE STERNAL,252.0,,Page-6_Line-21 : 0278 WIRE FLEXIBLE STERNAL #7 021221 2 252.00
1,Page-6_Line-22,0278,021221,HEMOSTAT ABSORBENT 4X8 I 021221 Z,536.0,,Page-6_Line-22 : 0278 |HEMOSTAT ABSORBENT 4X8 I 021221 Z 536.00
2,Page-6_Line-23,0278,021221,WASHER SUTURING CORONARY A4648,103.0,,Page-6_Line-23 : 0278 |WASHER SUTURING CORONARY A4648 021221 1 103.00
3,Page-6_Line-24,0278,021221,CATH MEDIASTINAL 9MM,282.0,,Page-6_Line-24 : 0278 |CATH MEDIASTINAL 9MM 1729 021221 2 282.00
4,Page-6_Line-25,0278,021221,PATCH TEFLON FELT CUSTOM,207.0,,Page-6_Line-25 : 0278 |PATCH TEFLON FELT CUSTOM (1768 021221 1 207.00


In [53]:
########### PDF UPLOAD AS INPUT #######################   

# s= '/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_uploads/CAMC_IB_Dana Keen_Claim # 21755930880.pdf'
s= '/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_uploads/Betsy Stewart_Claim # 21740462937.pdf'

########### PDF TO IMAGE PROCESSING #######################    
processed_image_loc, pdf_name = pdf_to_image(s)
print(processed_image_loc)
print(pdf_name)
########### IMAGE TO PDF PROCESSING #######################
processed_pdf = image_to_pdf(processed_image_loc, pdf_name)
print(processed_pdf)
########### PROCESSED PDF TO SEARCHABLE PDF #######################
clean_searchable_pdf_lines= PDF_searchable_lines(processed_pdf, pdf_name )


########### INPUT SEARCH TERM INTO SEARCHABLE PDF #######################
List_check=["0272", "250"]
search_word = "250"

cost_list_1, total_cost_1, search_df_1 = Search_in_PDF(search_word)
search_df_1

Betsy Stewart_Claim # 21740462937.pdf
Betsy Stewart_Claim # 21740462937
New File Transfer........
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/Betsy Stewart_Claim # 21740462937.pdf/Betsy Stewart_Claim # 21740462937_0.jpg
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/Betsy Stewart_Claim # 21740462937.pdf/Betsy Stewart_Claim # 21740462937_1.jpg
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/Betsy Stewart_Claim # 21740462937.pdf/Betsy Stewart_Claim # 21740462937_2.jpg
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/Betsy Stewart_Claim # 21740462937.pdf/Betsy Stewart_Claim # 21740462937_3.jpg
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/Betsy Stewart_Claim # 21740462937.pdf/Betsy Stewart_Claim # 21740462937_4.jpg
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/Betsy Stewart_Claim # 21740462937.pdf/Betsy Stewart_Claim # 21740462937_5.jpg
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/Betsy Stewart_

Scanning contents: 100%|███████████████████████| 8/8 [00:00<00:00, 118.60page/s]
OCR:  19%|██████▍                           | 1.5/8.0 [00:06<00:21,  3.37s/page][tesseract] lots of diacritics - possibly poor OCR
OCR: 100%|██████████████████████████████████| 8.0/8.0 [00:12<00:00,  1.58s/page]
PDF/A conversion: 100%|█████████████████████████| 8/8 [00:01<00:00,  5.99page/s]
JPEGs: 0image [00:00, ?image/s]

JBIG2: 0item [00:00, ?item/s]
PDF/A conversion: 100%|█████████████████████████| 8/8 [00:04<00:00,  1.74page/s]


[1101.0, 2.6, 1.0, 8.4, 0, 6.25, 26.5, 26.5, 15.5, 26.5, 26.5, 0, 56.5, 0, 9.09, 102.0, 56.5, 6.25, 1009.0, 0.025, 6.25, 72.0, 110.5, 56.5, 0, 0.025, 110.5, 0, 0, 110.5, 0.025, 0, 0, 0, 56.5, 0, 56.59, 6.25, 15.5, 110.5, 6.25, 72.0, 9.0, 56.5, 308.09, 2.69, 6.25, 19.85, 110.5, 0, 72.0, 0, 64.0, 6.25, 72.0, 0, 173208.9, 171000.8, 171000.8, 171000.8, 53599.0]
Total Amount :  743807.4349999999


,Position,Search_Term,Date,Description,Amount,FLAG,Line_info
0,Page-2_Line-39,250,06/17/20,IOHEXOL 6000MG/1000ML ORAL,1101.000,,Page-2_Line-39 : 06/17/20 0250 33514099 IOHEXOL 6000MG/1000ML ORAL 1101.00
1,Page-2_Line-44,250,0000002,ACETAMINOPHEN 2325MG TAB (QTY OF,2.600,,Page-2_Line-44 : 0€/17/20 0250 ACETAMINOPHEN 2325MG TAB (QTY OF 0000002) 2.60
2,Page-2_Line-49,250,06/17/20,W W EFACITRACIN OINT 30GM TOP,1.000,,Page-2_Line-49 : 06/17/20 0250 W W EFACITRACIN OINT 30GM TOP 1.00
3,Page-2_Line-54,250,0000002,MINERAL OIL 10 ML LIQ (QTY OF,8.400,,Page-2_Line-54 : 0€/17/20 0250 MINERAL OIL 10 ML LIQ (QTY OF 0000002) 8.40
4,Page-3_Line-13,250,06/18/20,INPATIENT VISIT - STAT,0.000,Check the Amount,Page-3_Line-13 : 06/18/20 0440 36250081 INPATIENT VISIT - STAT 0.00
5,Page-3_Line-25,250,06/18/20,DOCUSATE SODIUM 100MG CAP,6.250,,Page-3_Line-25 : 06/18/20 0250 33501222 DOCUSATE SODIUM 100MG CAP 6.25
6,Page-3_Line-27,250,510696,NORCO 5/325MG TABLET,26.500,,"Page-3_Line-27 : 0€/18/20 0250 3 510696 NORCO 5/325MG TABLET 26,50"
7,Page-3_Line-31,250,510696,NORCO 5/325MG TABLET,26.500,,"Page-3_Line-31 : 0€/18/20 0250 3 510696 NORCO 5/325MG TABLET 26,50"
8,Page-3_Line-37,250,06/18/20,LEVOTHYROXINE SOD 0.025MG,15.500,,Page-3_Line-37 : 06/18/20 0250 3 504139 LEVOTHYROXINE SOD 0.025MG 15.50
9,Page-3_Line-39,250,06/18/20,NORCO 5/325MG TABLET,26.500,,"Page-3_Line-39 : 06/18/20 0250 3 510696 NORCO 5/325MG TABLET 26,50"


In [55]:
########### PDF UPLOAD AS INPUT #######################   

# s= '/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_uploads/CAMC_IB_Dana Keen_Claim # 21755930880.pdf'
s= '/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_uploads/Betsy Stewart_Claim # 21740462937.pdf'

########### PDF TO IMAGE PROCESSING #######################    
processed_image_loc, pdf_name = pdf_to_image(s)
print(processed_image_loc)
print(pdf_name)
########### IMAGE TO PDF PROCESSING #######################
processed_pdf = image_to_pdf(processed_image_loc, pdf_name)
print(processed_pdf)
########### PROCESSED PDF TO SEARCHABLE PDF #######################
clean_searchable_pdf_lines= PDF_searchable_lines(processed_pdf, pdf_name )


########### INPUT SEARCH TERM INTO SEARCHABLE PDF #######################
List_check=["0272", "250"]
search_word = "250"

cost_list_1, total_cost_1, search_df_1 = Search_in_PDF(search_word)
search_df_1

Betsy Stewart_Claim # 21740462937.pdf
Betsy Stewart_Claim # 21740462937
New File Transfer........
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/Betsy Stewart_Claim # 21740462937.pdf/Betsy Stewart_Claim # 21740462937_0.jpg


/n01/data/nlp_aeac/mostofa_env/mo_venv/lib/python3.7/site-packages/ipykernel_launcher.py:474: RuntimeWarning: invalid value encountered in double_scalars
/n01/data/nlp_aeac/mostofa_env/mo_venv/lib/python3.7/site-packages/ipykernel_launcher.py:476: RuntimeWarning: invalid value encountered in double_scalars


126
NOISE REMOVAL DONE
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/Betsy Stewart_Claim # 21740462937.pdf/Betsy Stewart_Claim # 21740462937_1.jpg
126
NOISE REMOVAL DONE
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/Betsy Stewart_Claim # 21740462937.pdf/Betsy Stewart_Claim # 21740462937_2.jpg
126
NOISE REMOVAL DONE
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/Betsy Stewart_Claim # 21740462937.pdf/Betsy Stewart_Claim # 21740462937_3.jpg
127
NOISE REMOVAL DONE
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/Betsy Stewart_Claim # 21740462937.pdf/Betsy Stewart_Claim # 21740462937_4.jpg
127
NOISE REMOVAL DONE
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/Betsy Stewart_Claim # 21740462937.pdf/Betsy Stewart_Claim # 21740462937_5.jpg
127
NOISE REMOVAL DONE
/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_temp/Betsy Stewart_Claim # 21740462937.pdf/Betsy Stewart_Claim # 21740462937_6.jpg
126
NOISE REMOVAL DONE
/n01/data/n

Scanning contents: 100%|███████████████████████| 8/8 [00:00<00:00, 120.94page/s]
OCR: 100%|██████████████████████████████████| 8.0/8.0 [00:05<00:00,  1.59page/s]
PDF/A conversion: 100%|█████████████████████████| 8/8 [00:00<00:00,  7.51page/s]
JPEGs: 0image [00:00, ?image/s]

JBIG2: 0item [00:00, ?item/s]
PDF/A conversion: 100%|█████████████████████████| 8/8 [00:03<00:00,  2.50page/s]

[]
Total Amount :  0.0


,Position,Search_Term,Date,Description,Amount,FLAG,Line_info


In [17]:
print(processed_image_loc)
print(pdf_name)

/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/pdf_image_processed/IB_Claim # 21453296565.pdf
IB_Claim # 21453296565.pdf


In [19]:
import img2pdf


processed_pdf = image_to_pdf(processed_image_loc, pdf_name)

In [20]:
print(processed_pdf)

/n01/data/nlp_aeac/mostofa_env/ITEMIZED_BILL/image_processed_pdf/IB_Claim # 21453296565.pdf


In [24]:
clean_searchable_pdf_lines= PDF_searchable_lines(processed_pdf, pdf_name )

Scanning contents: 100%|█████████████████████| 19/19 [00:00<00:00, 277.16page/s]
OCR:   8%|██▌                              | 1.5/19.0 [00:12<02:10,  7.46s/page][tesseract] lots of diacritics - possibly poor OCR
OCR:  39%|█████████████                    | 7.5/19.0 [00:14<00:11,  1.02s/page][tesseract] lots of diacritics - possibly poor OCR
[tesseract] lots of diacritics - possibly poor OCR
OCR:  45%|██████████████▊                  | 8.5/19.0 [00:22<00:32,  3.09s/page][tesseract] lots of diacritics - possibly poor OCR
OCR:  55%|█████████████████▋              | 10.5/19.0 [00:25<00:17,  2.07s/page][tesseract] lots of diacritics - possibly poor OCR
[tesseract] lots of diacritics - possibly poor OCR
OCR:  66%|█████████████████████           | 12.5/19.0 [00:26<00:09,  1.39s/page][tesseract] lots of diacritics - possibly poor OCR
[tesseract] lots of diacritics - possibly poor OCR
OCR:  79%|█████████████████████████▎      | 15.0/19.0 [00:26<00:03,  1.27page/s][tesseract] lots of diacritics 

In [25]:
List_check=["0272", "250"]
search_word = "250"

cost_list_1, total_cost_1, search_df_1 = Search_in_PDF(search_word)
search_df_1

[14.85, 14.85, 40.5, 144.45, 64.45, 59.4, 74.85, 14.85, 4372.15, 0.9, 14.85, 122.35, 62.95, 59.4, 205.4, 1996.25, 74.85, 40.5, 14.85, 59.4, 122.35, 14.85, 14.85, 59.4, 0.3, 0, 0, 0, 0.9, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0.9, 0, 0, 1.0, 0, 0, 0.3, 0, 0, 0, 0, 0, 0, 0.9, 0, 0, 1.0, 0, 0, 0, 0.3, 0, 0, 0, 59.4, 122.35, 62.95, 74.85, 13.05, 122.35, 4.35, 40.5, 14.85, 4.3, 59.4, 59.4, 14.85, 62.95, 14.85, 59.4, 122.39, 62.95, 74.85, 13.05, 40.5, 4.35, 14.85, 4.3, 59.4, 122.35, 62.95, 14.85, 4.8, 62.95, 14.85, 59.4, 0.3, 4.8, 62.95, 74.85, 13.05, 40.5, 4.35, 14.85, 0, 0, 0, 0, 0, 0, 0, 0.3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 40.5, 4.35, 14.85, 71.2, 4.3, 122.35, 0, 165.6, 85.55, 79.35, 89.35, 0, 4.8, 122.35, 74.85, 13.05, 71.2, 40.5, 4.35, 14.85, 4.3, 122.35, 14.85, 71.2, 4.8, 122.35, 387.65, 74.85, 13.05, 40.5, 4.35, 62.95, 14.85, 71.2, 4.3, 122.35, 387.65, 62.95, 71.2, 0.3, 387.65, 62.95, 74.85, 13.05, 4.65, 4.3

,Position,Search_Term,Date,Description,Amount,FLAG,Line_info
0,Page-2_Line-52,250,010921,GABAPENTIN 400 MG CAP,14.8500,,Page-2_Line-52 : 0250 |GABAPENTIN 400 MG CAP 010921 2 14.85
1,Page-2_Line-54,250,010921,GABAPENTIN 400 MG CAP 010921 Z,14.8500,,Page-2_Line-54 : 0250 |GABAPENTIN 400 MG CAP 010921 Z 14.85
2,Page-2_Line-55,250,010921,O250 |ESCITALOPRAM 10 MG TAB,40.5000,,Page-2_Line-55 : O250 |ESCITALOPRAM 10 MG TAB 010921 2 40.50
3,Page-2_Line-56,250,010921,ENOXAPARIN 40 MG/0.4 ML,144.4500,,Page-2_Line-56 : 0250 |ENOXAPARIN 40 MG/0.4 ML 010921 1 144.45
4,Page-2_Line-57,250,010921,FUROSEMIDE 10 MG/ML 4 ML,64.4500,,"Page-2_Line-57 : 0250 |FUROSEMIDE 10 MG/ML 4 ML 010921 1 64,45"
5,Page-2_Line-58,250,010921,DEXAMETHASONE 10 MG/ML 1 010921 a,59.4000,,Page-2_Line-58 : 0250 |DEXAMETHASONE 10 MG/ML 1 010921 a 59.40
6,Page-2_Line-59,250,010921,PANTOPRAZOLE 40 MG VIAL,74.8500,,"Page-2_Line-59 : 0250 |PANTOPRAZOLE 40 MG VIAL, 010921 1 74.85"
7,Page-2_Line-61,250,010921,GABAPENTIN 400 MG CAP,14.8500,,Page-2_Line-61 : 0250 |GABAPENTIN 400 MG CAP 010921 2 14.85
8,Page-2_Line-62,250,010921,REMDESIVIR 100 MG LYPH P,4372.1500,,Page-2_Line-62 : 0250 |REMDESIVIR 100 MG LYPH P 010921 2 4372.15
9,Page-2_Line-63,250,010921,NACL 250 ML 010921 = Le7,0.9000,,Page-2_Line-63 : 0250 /0.9% NACL 250 ML 010921 = Le7 ~55


In [27]:

search_word = "0278"

cost_list_1, total_cost_1, search_df_1 = Search_in_PDF(search_word)
search_df_1

[]
Total Amount :  0.0


,Position,Search_Term,Date,Description,Amount,FLAG,Line_info
